<a href="https://colab.research.google.com/github/eyessoo/US_election_state_analysis/blob/main/US_election_state_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Problem Description 

You have been asked to help a candidate to become US president. The winner of the election will be the candidate winning the most grand electors. 

Grand electors are attributed at the state level: in each of the 51 states, there is a given number of grand electors to win (roughly, but not exactly, proportional to the size of the state) and the presidential candidate receiving the most local votes wins ALL the Grand Electors in that state.

Because the number of grand electors is not exactly proportional to the population, some states can be prioritized to increase the return on investment of the campaign. We assume here there are only 2 candidates, and no history (no trend of certain states to vote for a particular candidate or party). Hence, each vote is equally "expensive" to get, but some states grant more grand elector per capita.

You are provided with 2 tables: one giving the number of Grand Electors per state, the other the population per state.



---


## DATASET
Grand electors by State. [link](https://drive.google.com/file/d/1AuPlNXKjmEdRTLGqKbP-OP1XZ5UQPIPw/view)

Population by State. [link](https://drive.google.com/file/d/1VKt_hF2pRqPxcNb1DKotkVXWNd2HX_KL/view)



---



You are asked to identify the states that should be prioritized to win the election, with a smart but simple algorithm (brute force computation of all possibilities to find the absolute optimum is not accepted, as it would be to computationally expensive). It is ok not to find the overall optimum, we just want a strategy that makes sense.

We will rank states by decreasing number of grand electors per capita. The first states in the list will be the most valuable (you get a large number of grand electors by convincing a small number of people to vote for you). We will target all the states at the top of the list until the cumulative sum (also called running total) of grand electors won is larger than half the total number of Grand Electors in the country.

To do that, we need (you are allowed to create as many intermediary tables as you want, to keep queries short):

1. To join the 2 tables:
You notice States are not capitalized the same way in both tables (one is in uppercase letters, the other not), so you will first need to convert all to uppercase, for instance.
2. Now you can join the tables on the state key.
3. Your boss wants you to change the name of the "District of Columbia" state to its short version "DC". Please do that.
4. To compute the ratio between the number of grand electors and the population. Please create a new column with that ratio.
5. To order the states by decreasing ratio of Grand Electors per capita. That will make our priority list.
6. To compute the running total of Grand Electors in that sorted list.
Hint: you can get inspiration from here to compute a running total from here:  https://stackoverflow.com/questions/21382766/cumulative-summing-values-in-sqlite
Independently, to compute the half of the total of Grand Electors overall (in the whole country):
This is the threshold we need to reach for winning the presidential election.
7. To filter our sorted list of states in order to keep only the (top) ones enabling us to reach the computed threshold. (the other states can be ignored). That is our target list.

Hint: You can do that in 2 steps:
* Select all the states for which the running total is below or equal to the threshold.
* Add the first state for which the running total is larger than the threshold.


Can you draw some conclusions from the result? 

Is it in line with your expectations? 

How many states do you end up with in the target list? 

Is it a small or a large number? Do you think it would be a good recommendation to target those states?

In [1]:
# We will first load the sql extension into our environment
#
%load_ext sql

# Then connect to our in memory sqlite database
# NB: This database will cease to exist as soon as the database connection is closed
#
%sql sqlite://

'Connected: @None'

In [2]:
# Importing the python libraries that we will need in this notebook
#
import csv
import pandas as pd

In [3]:
with open('GrandElectors_by_state.csv','r') as f:
    GrandElectors = pd.read_csv(f, index_col=0, encoding='utf-8') 

# Then storing it in an SQL table of our in memory sqlite database
%sql DROP TABLE if EXISTS GrandElectors;
%sql PERSIST GrandElectors;

 * sqlite://
Done.
 * sqlite://


'Persisted grandelectors'

In [4]:
with open('Population_by_state.csv','r') as f:
    Population = pd.read_csv(f, index_col=0, encoding='utf-8') 

# Then storing it in an SQL table of our in memory sqlite database
%sql DROP TABLE if EXISTS Population;
%sql PERSIST Population;

 * sqlite://
Done.
 * sqlite://


'Persisted population'

# Data Understanding


In [5]:
%%sql
SELECT * FROM GrandElectors LIMIT 5;

 * sqlite://
Done.


State,GrandElectors
Alabama,9
Alaska,3
Arizona,11
Arkansas,6
California,55


In [6]:
%%sql
SELECT * FROM Population LIMIT 5;

 * sqlite://
Done.


State,Population
CALIFORNIA,39144818
TEXAS,27469114
FLORIDA,20271272
NEW YORK,19795791
ILLINOIS,12859995


# Data Preparation

In [7]:
#You notice States are not capitalized the same way in both tables (one is in uppercase letters, the other not), 
# so you will first need to convert all to uppercase, for instance.

%%sql

UPDATE GrandElectors
SET State = UPPER(state)

 * sqlite://
51 rows affected.


[]

In [8]:
%%sql
SELECT * FROM GrandElectors LIMIT 5;

 * sqlite://
Done.


State,GrandElectors
ALABAMA,9
ALASKA,3
ARIZONA,11
ARKANSAS,6
CALIFORNIA,55


In [9]:
#Now you can join the tables on the state key.

%%sql

CREATE VIEW joined
AS
SELECT GrandElectors.State, GrandElectors, Population
FROM GrandElectors INNER JOIN Population
ON GrandElectors.State = Population.State

 * sqlite://
Done.


[]

In [10]:
%%sql

SELECT * from joined

 * sqlite://
Done.


State,GrandElectors,Population
ALABAMA,9,4858979
ALASKA,3,738432
ARIZONA,11,6828065
ARKANSAS,6,2978204
CALIFORNIA,55,39144818
COLORADO,9,5456574
CONNECTICUT,7,3590886
DISTRICT OF COLUMBIA,3,672228
DELAWARE,3,945934
FLORIDA,29,20271272


In [11]:
#Your boss wants you to change the name of the "District of Columbia" state to its short version "DC". Please do that.

%%sql
UPDATE GrandElectors
SET State = "DC"
WHERE State = "DISTRICT OF COLUMBIA"

 * sqlite://
1 rows affected.


[]

In [12]:
%%sql

SELECT * FROM GrandElectors Limit 10;

 * sqlite://
Done.


State,GrandElectors
ALABAMA,9
ALASKA,3
ARIZONA,11
ARKANSAS,6
CALIFORNIA,55
COLORADO,9
CONNECTICUT,7
DC,3
DELAWARE,3
FLORIDA,29


In [13]:
%%sql
UPDATE Population
SET State = "DC"
WHERE State = "DISTRICT OF COLUMBIA"

 * sqlite://
1 rows affected.


[]

In [14]:
%%sql

SELECT * FROM Population
ORDER BY State ASC Limit 10;

 * sqlite://
Done.


State,Population
ALABAMA,4858979
ALASKA,738432
AMERICAN SAMOA,54343
ARIZONA,6828065
ARKANSAS,2978204
CALIFORNIA,39144818
COLORADO,5456574
CONNECTICUT,3590886
DC,672228
DELAWARE,945934


In [15]:
#Now you can join the tables on the state key.

%%sql

CREATE VIEW joined_update
AS
SELECT GrandElectors.State, GrandElectors, Population
FROM GrandElectors INNER JOIN Population
ON GrandElectors.State = Population.State

 * sqlite://
Done.


[]

In [16]:
%%sql
SELECT * FROM joined_update LIMIT 10;

 * sqlite://
Done.


State,GrandElectors,Population
ALABAMA,9,4858979
ALASKA,3,738432
ARIZONA,11,6828065
ARKANSAS,6,2978204
CALIFORNIA,55,39144818
COLORADO,9,5456574
CONNECTICUT,7,3590886
DC,3,672228
DELAWARE,3,945934
FLORIDA,29,20271272


In [17]:
# To compute the ratio between the number of grand electors and the population. Please create a new column with that ratio.
# To order the states by decreasing ratio of Grand Electors per capita. 

%%sql

CREATE VIEW Ratios
AS
SELECT State, GrandElectors, Population, (GrandElectors *1.0/Population *1.0) as Ratio
FROM joined_update
ORDER BY Ratio DESC;



 * sqlite://
Done.


[]

In [18]:
%%sql

SELECT * FROM Ratios
LIMIT 10;

 * sqlite://
Done.


State,GrandElectors,Population,Ratio
WYOMING,3,586107,5.118519314732634e-06
VERMONT,3,626042,4.792010759661493e-06
DC,3,672228,4.462771559649405e-06
ALASKA,3,738432,4.0626625065002604e-06
NORTH DAKOTA,3,756927,3.963394092164766e-06
RHODE ISLAND,4,1056298,3.7868101615263876e-06
SOUTH DAKOTA,3,858469,3.4945932817609025e-06
DELAWARE,3,945934,3.1714686225466047e-06
MAINE,4,1329328,3.009039153617467e-06
NEW HAMPSHIRE,4,1330608,3.0061445594795764e-06


In [19]:
#To compute the running total of Grand Electors in that sorted list.

%%sql
CREATE VIEW
Sum
AS
SELECT t1.*, sum(t2.GrandElectors) as RunningSum
FROM Ratios t1 INNER JOIN Ratios t2 
ON t1.Ratio <= t2.Ratio
GROUP BY t1.State
ORDER BY t1.Ratio ASC;


 * sqlite://
Done.


[]

In [20]:
%%sql

SELECT * FROM SUM;

 * sqlite://
Done.


State,GrandElectors,Population,Ratio,RunningSum
TEXAS,38,27469114,1.3833718845099992e-06,538
CALIFORNIA,55,39144818,1.4050391037710278e-06,500
FLORIDA,29,20271272,1.4305959685213637e-06,445
NEW YORK,29,19795791,1.4649578791774474e-06,416
NORTH CAROLINA,15,10042802,1.4936070630487388e-06,387
OHIO,18,11613423,1.5499306276883225e-06,372
VIRGINIA,13,8382993,1.5507587803067473e-06,354
ILLINOIS,20,12859995,1.5552105580134362e-06,341
PENNSYLVANIA,20,12802503,1.562194517743913e-06,321
NEW JERSEY,14,8958013,1.562846582160575e-06,301


In [21]:
# Filtering out the states that have a running sum of 269 +1

%%sql
SELECT Sum.State from Sum
WHERE RunningSum <= 271
ORDER BY Sum.Ratio DESC;

 * sqlite://
Done.


State
WYOMING
VERMONT
DC
ALASKA
NORTH DAKOTA
RHODE ISLAND
SOUTH DAKOTA
DELAWARE
MAINE
NEW HAMPSHIRE


# Recommendation

Candidates should focus their efforts on the states filtered above as it will
increase their chances of winning the election.